<a href="https://colab.research.google.com/github/greyhound101/shopee/blob/main/resnet50_m3d5_dynamic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [4]:
import shutil
shutil.copy( '/content/gdrive/My Drive/shopee/train_images.zip','/content/')

'/content/train_images.zip'

In [5]:
import os
try:
  os.mkdir('/content/train images')
except:
  pass

In [6]:
import zipfile
a=zipfile.ZipFile('/content/train_images.zip', 'r')
from tqdm import tqdm
while True:
  present=os.listdir('/content/train images')
  
  for file in tqdm(a.namelist()):
    if file not in present:
      a.extract(file,'/content/train images')
  if len(set(present)-set(a.namelist()))==0:
      break

100%|██████████| 32412/32412 [00:45<00:00, 706.38it/s]


In [7]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math
from torch.nn.parameter import Parameter
import torchvision.models as models
from torch.autograd import Variable
import torch
import torch.nn.functional as F
import math
from torch import nn
from torch.nn.parameter import *


NUMBER_OF_CLASSES = 11014


class ArcFaceLoss(nn.modules.Module):
    def __init__(self,device, s=45.0, crit="bce", reduction="mean"):
        super().__init__()

        self.reduction = reduction
        
        self.crit = nn.CrossEntropyLoss(reduction="mean")   

        self.s = s

    def forward(self,m, logits, labels):
        
        self.cos_m = torch.cos(m)
        self.sin_m = torch.sin(m)
        self.th = torch.cos(math.pi - m)
        self.mm = torch.sin(math.pi - m) * m
        self.device=device
        logits = logits.float()
        cosine = logits
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        
        labels2 = torch.zeros_like(cosine)
        labels2.scatter_(1, labels.view(-1, 1).long(), 1)
        output = (labels2 * phi) + ((1.0 - labels2) * cosine)
        self.s = torch.nn.Parameter(torch.tensor([45.], requires_grad=False)).to(self.device)

        output = output * self.s
        loss = self.crit(output, labels)
        
        return loss
          
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, features):
        cosine = F.linear(F.normalize(features), F.normalize(self.weight))
        return cosine

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=True):
        super(GeM,self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1)*p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
   
class Backbone(nn.Module):

    
    def __init__(self, name='resnet50', pretrained=True):
        super(Backbone, self).__init__()
        self.net = nn.Sequential(*list( models.resnet50().children())[:-2])
        self.out_features = 2048
    def forward(self, x):
        x = self.net(x)

        return x

    
class Net(nn.Module):
    def __init__(self, pretrained=True):
        super(Net, self).__init__()
        
        self.backbone = Backbone()
        
        self.global_pool = GeM(p_trainable=3)
        self.embedding_size = 256      
        
        self.neck = nn.Sequential(
                nn.Linear(self.backbone.out_features, self.embedding_size, bias=True),
                nn.BatchNorm1d(self.embedding_size),
                torch.nn.PReLU()
            )
            
        self.head = ArcMarginProduct(self.embedding_size, NUMBER_OF_CLASSES)
        
        
    def forward(self, x):

        x = self.backbone(x)
        x = self.global_pool(x)
        x = x[:,:,0,0]
        
        x = self.neck(x)

        logits = self.head(x)
        
        return logits

In [20]:
import numpy as np
import torch
import albumentations as A
from PIL import Image, ImageFile
class DatasetRetriever:
    def __init__(self, X,margin, y):
        self.X=X
        self.margin=margin
        self.root='/content/train images/'
        self.y = y
        self.aug = A.Compose(
                [
                    A.Resize(512,512),
                    A.RandomCrop(448,448),
                    A.Normalize()
                ]
            )


    def __len__(self):
        return self.X.shape[0]
        
    def __getitem__(self,idx):
        path = self.X[idx]
        target=self.y[idx]
        margin=self.margin[idx]
        image_path = self.root+path
        image = Image.open(image_path)
        target = self.y[idx]

        image = np.array(image)
        augmented = self.aug(image=image)
        image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)

        return image, torch.tensor(target, dtype=torch.long),margin

In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df=pd.read_csv('/content/gdrive/MyDrive/shopee/train.csv.zip')
df['counts']=df['label_group'].map(df['label_group'].value_counts())
df['counts'] = np.sqrt(1 / np.sqrt(df['counts'].values))
df['margin'] = (df['counts'] - min(df['counts'])) / (max(df['counts']) - min(df['counts'])) * 0.45 + 0.05
le=LabelEncoder()
df['label_group']=le.fit_transform(df['label_group'])
df.head()

,posting_id,image,image_phash,title,label_group,counts,margin
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,666,0.840896,0.5
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",7572,0.840896,0.5
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,6172,0.840896,0.5
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,10509,0.840896,0.5
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,9425,0.840896,0.5


In [23]:
def train(train_loader,valid_loader, model, device, optimizer):
    model.train()
    t = tqdm(train_loader, position=0, leave=True)
    total_loss=[]
    for step, d in enumerate(t):
#         if step<2:
         image = d[0].to(device)
         target = d[1].to(device)
         margin = d[2].view(-1,1).to(device)
         model.zero_grad()
         logits = model(image)
         loss = ArcFaceLoss(device=device)(logits=logits, labels=target,m=margin)
         t.set_description(str(loss.item()))
         total_loss.append(loss.item())
         loss.backward()
         optimizer.step()
#         else:
#             break
    val_loss=[]
    t = tqdm(valid_loader, position=0, leave=True)   
    for step, d in enumerate(t):
#         if step<2:
         image = d[0].to(device)
         target = d[1].to(device)
         margin = d[2].view(-1,1).to(device)
         logits = model(image)
         loss = ArcFaceLoss(device=device)(logits=logits, labels=target,m=margin)
         val_loss.append(loss.item())
#         else:
#             break
         
    return total_loss,val_loss

In [24]:
from sklearn.model_selection import train_test_split
trn,val=train_test_split(df,test_size=0.35, random_state=42,stratify = df['label_group'])

In [25]:
rem=list(set(val['label_group'].unique())-set(trn['label_group'].unique()))
rem=val.loc[val['label_group'].isin(rem)]
val=val.loc[~(val['label_group'].isin(rem))]
trn=pd.concat([trn,rem],0)
print(trn.shape,val.shape,len(list(set(val['label_group'].unique())-set(trn['label_group'].unique()))))

(22262, 7) (11988, 7) 0


In [26]:
from tqdm import tqdm
from statistics import *
model=Net()
optimizer = torch.optim.Adam(model.parameters(), 
                            lr=5e-4,weight_decay =1e-5)
train_dataset = DatasetRetriever(
        X = trn['image'].values,margin=trn['margin'].values,
        y = trn['label_group'].values)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16,
            drop_last=True
        )

valid_dataset = DatasetRetriever(
        X = val['image'].values,margin=val['margin'].values,
        y = val['label_group'].values)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=16,
            drop_last=True
        )


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:


model.to(device)
trn_loss=[]
val_loss=[]

for i in range(20):
    t,v=train(train_loader,valid_loader, model, device, optimizer)
    trn_loss.append(mean(t))
    val_loss.append(mean(v))
    



28.64346385671967:   7%|▋         | 101/1391 [00:58<12:08,  1.77it/s] 

In [ ]:
from matplotlib import pyplot as plt
plt.plot(trn_loss)

In [ ]:
plt.plot(val_loss)

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/shopee/resnet50dm5.hdf5')